# Processar horas

In [1]:
import numpy as np
import pandas as pd
import ast

In [2]:
# declare a list tasks whose products you want to use as inputs
upstream = None
product = None
discrete_hours = None # valida se as horas serão discretas ou contínuas 
df_input_path = '../../dados/X_trainToronto.csv'

In [3]:
# Parameters
df_input_path = "../../dados/X_trainToronto.csv"
discrete_hours = True
product = {
    "data": "/home/guilherme/\xc1rea de Trabalho/Estudo Faculdade/Disciplinas/minera\xe7\xe3o de dados/Desafio/pipelines/processamento_features/output/processar_hours_data.parquet",
    "nb": "/home/guilherme/\xc1rea de Trabalho/Estudo Faculdade/Disciplinas/minera\xe7\xe3o de dados/Desafio/pipelines/processamento_features/output/output_processar_hours.ipynb",
}


In [4]:
def literal_eval(x):
    if pd.isna(x):
        return dict()
    raw_dict = ast.literal_eval(x)
    for key in raw_dict:
        raw_dict[key] = ast.literal_eval(raw_dict[key])
    return raw_dict
        
def proccess_df(df):
    df['hours'] = df['hours'].apply(lambda x: ast.literal_eval(x) if not pd.isna(x) else dict())

In [5]:
df_reg = pd.read_csv(df_input_path)
proccess_df(df_reg)
df_reg = df_reg[['business_id', 'hours']]
df_reg

,business_id,hours
0,vHzWmPWHN4J1hRR3W3AMQg,"{'Monday': '18:0-23:0', 'Tuesday': '18:0-23:0'..."
1,15to24Q-otAHmto7FzsWRg,"{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."
2,8aqKdf4G4AAir8k_Kdslvg,"{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
3,uxU1vr5AhhkTQ83X0bpeyg,{}
4,f702hTJoqdR34Jn23C7d1A,{}
...,...,...
17577,Zp5xil8pC6Bm-fFOWIbh0Q,"{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
17578,o2811xZ0ufRi7WcsqWiGzw,{}
17579,v9T9m_TbpLuaC2Kud5UbLA,"{'Monday': '10:0-18:0', 'Tuesday': '10:0-18:0'..."
17580,bzGnAIe9OiBHGWJE-dZ_8g,"{'Monday': '11:30-1:0', 'Tuesday': '11:30-1:0'..."


## Tratamento das horas abertas

In [6]:
from datetime import datetime, timedelta

midnight_tomorrow = datetime.strptime("0:0", "%H:%M") + timedelta(days=1)
midnight_today = datetime.strptime("0:0", "%H:%M")
daysOfWeek = {'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'}
times_of_day = {
         'morning': (datetime.strptime('6:0', "%H:%M"), datetime.strptime('12:0', "%H:%M")), 
         'afternoon': (datetime.strptime('12:0', "%H:%M"), datetime.strptime('18:0', "%H:%M")), 
         'evening': (datetime.strptime('18:0', "%H:%M"), midnight_tomorrow),
         'night': (midnight_today, datetime.strptime('6:0', "%H:%M"))
        }

def fill_hours(obj):
    result = {}
    for dayOfweek in daysOfWeek:
        hour1 = datetime.strptime("0:0", "%H:%M")
        hour2 = datetime.strptime("0:0", "%H:%M")
        if dayOfweek in obj:
            hours_obj = obj[dayOfweek].split('-')
            hour1 = datetime.strptime(hours_obj[0], "%H:%M")
            hour2 = datetime.strptime(hours_obj[1], "%H:%M")
            if hour1 >= hour2:
                hour2 += timedelta(days=1)
        for key in times_of_day:
            d1 = times_of_day[key][0]
            d2 = times_of_day[key][1]
            
            value = calculate_hoursOfDay(hour1, hour2, d1, d2)
            result[f'business_open_{dayOfweek}_{key}'] = value
    return result
            
    
def calculate_hoursOfDay(hour1: datetime, hour2: datetime, d1: datetime, d2: datetime):
    if hour1 > d2 or hour2 < d1:
        if hour2.day == midnight_tomorrow.day and (hour1 != midnight_tomorrow and hour2 != midnight_tomorrow):
            day1 = calculate_hoursOfDay(midnight_tomorrow, hour2, d1 + timedelta(days=1), d2 + timedelta(days=1))
            day2 = calculate_hoursOfDay(hour1, midnight_tomorrow, d1, d2)
            return day1 + day2
        return 0.0
        
    inf = hour1
    sup = hour2
    if hour1 <= d1:
        inf = d1
    if hour2 >= d2:
        sup = d2
        
    if discrete_hours == True and float((sup - inf).seconds) > 0.001:
        return 1.0
    return float((sup - inf).seconds) / 21600.0

In [7]:
hours_features = pd.DataFrame(list(df_reg['hours'].apply(fill_hours)))
df_reg = pd.concat([df_reg, hours_features], axis=1)
df_reg

,business_id,hours,business_open_Friday_morning,business_open_Friday_afternoon,business_open_Friday_evening,business_open_Friday_night,business_open_Monday_morning,business_open_Monday_afternoon,business_open_Monday_evening,business_open_Monday_night,...,business_open_Tuesday_evening,business_open_Tuesday_night,business_open_Thursday_morning,business_open_Thursday_afternoon,business_open_Thursday_evening,business_open_Thursday_night,business_open_Sunday_morning,business_open_Sunday_afternoon,business_open_Sunday_evening,business_open_Sunday_night
0,vHzWmPWHN4J1hRR3W3AMQg,"{'Monday': '18:0-23:0', 'Tuesday': '18:0-23:0'...",0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,15to24Q-otAHmto7FzsWRg,"{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ...",1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8aqKdf4G4AAir8k_Kdslvg,"{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,uxU1vr5AhhkTQ83X0bpeyg,{},0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,f702hTJoqdR34Jn23C7d1A,{},0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17577,Zp5xil8pC6Bm-fFOWIbh0Q,"{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
17578,o2811xZ0ufRi7WcsqWiGzw,{},0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17579,v9T9m_TbpLuaC2Kud5UbLA,"{'Monday': '10:0-18:0', 'Tuesday': '10:0-18:0'...",1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
17580,bzGnAIe9OiBHGWJE-dZ_8g,"{'Monday': '11:30-1:0', 'Tuesday': '11:30-1:0'...",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [8]:
df_reg.to_parquet(product['data'])

## Teste manual

In [9]:
hours_columns = [x for x in df_reg.columns if x.startswith('business_open_')]
from IPython.display import display
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(df_reg.iloc[13]['hours'])
    display(df_reg[hours_columns].iloc[[13]])
    display('----------------------------------')
    display(df_reg.iloc[542]['hours'])
    display(df_reg[hours_columns].iloc[[542]])
    display('----------------------------------')
    display(df_reg.iloc[5389]['hours'])
    display(df_reg[hours_columns].iloc[[5389]])

{'Monday': '7:0-17:0',
 'Tuesday': '6:0-19:0',
 'Wednesday': '6:0-19:0',
 'Thursday': '6:0-19:0',
 'Friday': '6:0-19:0',
 'Saturday': '5:0-18:0',
 'Sunday': '8:0-17:0'}

,business_open_Friday_morning,business_open_Friday_afternoon,business_open_Friday_evening,business_open_Friday_night,business_open_Monday_morning,business_open_Monday_afternoon,business_open_Monday_evening,business_open_Monday_night,business_open_Wednesday_morning,business_open_Wednesday_afternoon,business_open_Wednesday_evening,business_open_Wednesday_night,business_open_Saturday_morning,business_open_Saturday_afternoon,business_open_Saturday_evening,business_open_Saturday_night,business_open_Tuesday_morning,business_open_Tuesday_afternoon,business_open_Tuesday_evening,business_open_Tuesday_night,business_open_Thursday_morning,business_open_Thursday_afternoon,business_open_Thursday_evening,business_open_Thursday_night,business_open_Sunday_morning,business_open_Sunday_afternoon,business_open_Sunday_evening,business_open_Sunday_night
13,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0


'----------------------------------'

{'Monday': '9:30-19:0',
 'Tuesday': '9:30-19:0',
 'Wednesday': '9:30-19:0',
 'Thursday': '9:30-19:0',
 'Friday': '9:30-19:0',
 'Saturday': '9:30-18:0',
 'Sunday': '11:0-17:0'}

,business_open_Friday_morning,business_open_Friday_afternoon,business_open_Friday_evening,business_open_Friday_night,business_open_Monday_morning,business_open_Monday_afternoon,business_open_Monday_evening,business_open_Monday_night,business_open_Wednesday_morning,business_open_Wednesday_afternoon,business_open_Wednesday_evening,business_open_Wednesday_night,business_open_Saturday_morning,business_open_Saturday_afternoon,business_open_Saturday_evening,business_open_Saturday_night,business_open_Tuesday_morning,business_open_Tuesday_afternoon,business_open_Tuesday_evening,business_open_Tuesday_night,business_open_Thursday_morning,business_open_Thursday_afternoon,business_open_Thursday_evening,business_open_Thursday_night,business_open_Sunday_morning,business_open_Sunday_afternoon,business_open_Sunday_evening,business_open_Sunday_night
542,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0


'----------------------------------'

{'Monday': '9:0-21:0',
 'Tuesday': '9:0-21:0',
 'Wednesday': '9:0-21:0',
 'Thursday': '9:0-21:0',
 'Friday': '9:0-21:0',
 'Saturday': '9:0-20:0',
 'Sunday': '10:0-18:0'}

,business_open_Friday_morning,business_open_Friday_afternoon,business_open_Friday_evening,business_open_Friday_night,business_open_Monday_morning,business_open_Monday_afternoon,business_open_Monday_evening,business_open_Monday_night,business_open_Wednesday_morning,business_open_Wednesday_afternoon,business_open_Wednesday_evening,business_open_Wednesday_night,business_open_Saturday_morning,business_open_Saturday_afternoon,business_open_Saturday_evening,business_open_Saturday_night,business_open_Tuesday_morning,business_open_Tuesday_afternoon,business_open_Tuesday_evening,business_open_Tuesday_night,business_open_Thursday_morning,business_open_Thursday_afternoon,business_open_Thursday_evening,business_open_Thursday_night,business_open_Sunday_morning,business_open_Sunday_afternoon,business_open_Sunday_evening,business_open_Sunday_night
5389,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
